[@tsatie](https://x.com/tsatie) さんのあみだくじのコードを高速化してみました。

https://github.com/tsatie/SatieGitHubTest/blob/master/20250426AmidaCount.ipynb

あまり良くは理解していませんが，ChatGPT 4o-mini-highを用いました。

ポイントもまとめてもらいました。


以下が高速化のキモとなるポイントです。

1.	全列挙→DP（動的計画法）への置き換え
	- 元コードは長さ L=2a+n のビット列を全パターン 2^L 生成していたが、DP では「現在の高さ」と「現在の並び（perm）」の状態ごとに通り数をまとめて管理。
	- 各ステップで状態数分の遷移を行うだけなので、実質的には O(L \times \text{\#状態}) の計算量に抑えられる。
2.	状態の要約： (高さ, perm)
	- 高さ（現在の累積和）と、縦線の並び替え結果を表す perm（長さ n のベクトル）をキーにしてカウントを保持。
	- 同じ状態が重複して現れたら、その都度再計算せずにカウントを足し合わせることで枝刈りと重複除去を同時に実現。
3.	遷移の単純化
	- 「上へ（+1）」と「下へ（−1）＋スワップ」の２種類だけを考慮。
	- 各状態から最大２つの遷移しか呼び出さないため、１ステップあたりの計算が極めて軽量。
4.	最終ステップだけ特別扱い
	- 最後の１ステップは「高さが n-1 → +1 して n になるもの」を一括で集計。
	- 余分な状態を生成せず、集計ループも一回で済む。
5.	オンラインでの統計更新
	- 最終状態ごとにそのまま stat[i, perm[i]+1] を累積。
	- 別途、全パターンを列挙してから置換を適用する必要がなく、メモリ／時間の両面で効率的。

⸻

これらの工夫により、元の 2^{28} 程度の全探索が瞬時に終了するようになりました。


＜以下は(@tsatie)[https://x.com/tsatie] さんのコード作成の説明です＞

# 標準的な「あみだくじ」でどの筋を選ぶとどの筋に至るかを数える
### Python で組んだものを Claude にお願いして Julia に書き換えて貰った
数え方の要領は, 「あみだくじ」は「行きつ戻りつ」と1:1に対応する（らしい）事を利用している。

例えば, 筋は5本で横線が4本とすれば, $5+4\times2−1$ 回の移動で, 筋から隣の筋への移動をして, 左端から右端まで動くことを「行きつ戻りつ」と表現しよう。
筋を左から 0,1,2,3,4 とでもすれば, 例えば 0→1→0→1→0→1→0→1→0→1→2→3→4 のように5つの筋を「行きつ戻りつ」して最終的に到達するということ。
確かに「戻る」動き(この例では1→0しかない)が4回あることが確認できる。そしてこの「戻る」動きがあみだくじの置換に対応するというアイデアなのだ。

プログラムの概要は以下の通り。筋の数は tn 横棒の数は an としている。
+ 「行きつ戻りつ」を探るために2進数を総当たりして, 行きつ戻りつで辿り着くものを拾う
  - 先ず2進数を順に当たる
  - 文字列化して 0,1 の配列に直し 各項を2倍して1引くことで, -1,1の配列にする
  - その上で配列の先頭からの部分和が 0以上で tn 以下であるもので, 全体和が tn となる（つまり最後では端に到達する）ものを拾う
+ 拾えたらその「行きつ戻りつ」でどの順に筋を経由するかのリストを作成してそこから「あみだくじ」の置換を拾う
  - 「行きつ戻りつ」の行程で「戻る」ときにその2筋を交換していることになる
  - 筋の経由リストから「戻る」部分をピックアップして, 筋の置換を列挙した配列を作る
  - できた置換の配列で順に置換することで, 選んだ筋と結果到達する筋のリストを作成する
+ 「選んだ筋」と「到達する筋」の2次元配列に該当するものを足していく
+ 最終的に分布が得られる。

In [ ]:
function calculate_amidakuji_distribution(tn, an)
    # tn: number of vertical lines, an: number of horizontal bars
    ln = 2*an + tn  # total steps

    # Initialize DP: map (height, perm) to count of ways
    dp = Dict{Tuple{Int, Vector{Int}}, Int}()
    dp[(0, collect(0:tn-1))] = 1

    # Iterate through the first ln-1 steps
    for step in 1:ln-1
        new_dp = Dict{Tuple{Int, Vector{Int}}, Int}()
        for ((h, perm), c) in dp
            # Up step (+1)
            if h + 1 <= tn - 1
                key_up = (h + 1, perm)
                new_dp[key_up] = get(new_dp, key_up, 0) + c
            end
            # Down step (-1) and swap if h >= 1
            if h >= 1
                newh = h - 1
                perm2 = copy(perm)
                # Swap columns h and h+1 (1-indexed swap of perm entries)
                perm2[h], perm2[h+1] = perm2[h+1], perm2[h]
                key_dn = (newh, perm2)
                new_dp[key_dn] = get(new_dp, key_dn, 0) + c
            end
        end
        dp = new_dp
    end

    # Final step: only +1 moves that end at height == tn
    stat = zeros(Int, tn, tn)
    total = 0
    for ((h, perm), c) in dp
        if h == tn - 1
            # Apply final +1 => final height = tn
            total += c
            for i in 1:tn
                stat[i, perm[i]+1] += c
            end
        end
    end

    # Output
    println("--start---------------------------------------")
    println("$tn $an")
    println("--Ans-------------------------------------------")
    println(total)
    for i in 1:tn
        println(stat[i, :])
    end
    println("--end-------------------------------------------")
end


calculate_amidakuji_distribution (generic function with 1 method)

In [24]:
calculate_amidakuji_distribution(8, 10)

--start---------------------------------------
8 10
--Ans-------------------------------------------
1282735
[764877, 279584, 133631, 64604, 27257, 9481, 2693, 608]
[279584, 478114, 262307, 147260, 72988, 29809, 9980, 2693]
[133631, 262307, 365985, 252938, 153368, 75216, 29809, 9481]
[64604, 147260, 252938, 314118, 250202, 153368, 72988, 27257]
[27257, 72988, 153368, 250202, 314118, 252938, 147260, 64604]
[9481, 29809, 75216, 153368, 252938, 365985, 262307, 133631]
[2693, 9980, 29809, 72988, 147260, 262307, 478114, 279584]
[608, 2693, 9481, 27257, 64604, 133631, 279584, 764877]
--end-------------------------------------------


In [25]:
calculate_amidakuji_distribution(8,12)

--start---------------------------------------
8 12
--Ans-------------------------------------------
16131656
[9188341, 3508269, 1778834, 939616, 451633, 184261, 63000, 17702]
[3508269, 5568480, 3238722, 1961381, 1086206, 507952, 197646, 63000]
[1778834, 3238722, 4168532, 3074842, 2048283, 1130230, 507952, 184261]
[939616, 1961381, 3074842, 3550353, 3019342, 2048283, 1086206, 451633]
[451633, 1086206, 2048283, 3019342, 3550353, 3074842, 1961381, 939616]
[184261, 507952, 1130230, 2048283, 3074842, 4168532, 3238722, 1778834]
[63000, 197646, 507952, 1086206, 1961381, 3238722, 5568480, 3508269]
[17702, 63000, 184261, 451633, 939616, 1778834, 3508269, 9188341]
--end-------------------------------------------


In [26]:
calculate_amidakuji_distribution(8,20)

--start---------------------------------------
8 20
--Ans-------------------------------------------
393151913464
[189723516102, 83314065040, 49329548957, 31599142379, 19875091516, 11372795389, 5626371851, 2311382230]
[83314065040, 105606954285, 72716790819, 52653359557, 37020754687, 23496158993, 12717458232, 5626371851]
[49329548957, 72716790819, 77637484780, 66208709033, 53600482213, 38789943280, 23496158993, 11372795389]
[31599142379, 52653359557, 66208709033, 68259312069, 63935062010, 53600482213, 37020754687, 19875091516]
[19875091516, 37020754687, 53600482213, 63935062010, 68259312069, 66208709033, 52653359557, 31599142379]
[11372795389, 23496158993, 38789943280, 53600482213, 66208709033, 77637484780, 72716790819, 49329548957]
[5626371851, 12717458232, 23496158993, 37020754687, 52653359557, 72716790819, 105606954285, 83314065040]
[2311382230, 5626371851, 11372795389, 19875091516, 31599142379, 49329548957, 83314065040, 189723516102]
--end-------------------------------------------